In [1]:
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as ipw

from lib.dataset_wrapper import Dataset
from lib import abx_utils
from lib import notebooks

In [2]:
DATASET_NAME = "pb2007"

dataset = Dataset(DATASET_NAME)
assert dataset.has_palate
items_ema = dataset.get_items_data("ema")

In [3]:
LIPS_CONSONANTS = ["p", "b"]
TONGUE_CONSONANTS = ["t", "d", "k", "g"]
TESTED_CONSONANTS = LIPS_CONSONANTS + TONGUE_CONSONANTS

vowels = dataset.phones_infos["vowels"]
dataset_lab = {DATASET_NAME: dataset.lab}
palate = dataset.palate

In [4]:
consonants_indexes = abx_utils.get_datasets_phones_indexes(
    dataset_lab, TESTED_CONSONANTS, vowels
)

In [5]:
consonants_metrics = {}

for consonant in TESTED_CONSONANTS:
    consonant_metrics = {
        "min_lips_distance": [],
        "min_lips_ema": [],
        "min_tongue_tip_distance": [],
        "min_tongue_tip_ema": [],
        "min_tongue_mid_distance": [],
        "min_tongue_mid_ema": [],
    }
    consonants_metrics[consonant] = consonant_metrics
    
    for consonant_index in consonants_indexes[consonant]:
        consonant_ema = items_ema[consonant_index[1]][consonant_index[2]]
        nb_frames = len(consonant_ema)
        
        lower_lip_ema = consonant_ema[:, 8:10]
        upper_lip_ema = consonant_ema[:, 10:12]
        lips_distance = np.sqrt(np.sum((lower_lip_ema - upper_lip_ema) ** 2, axis=1))
        min_lips_distance = lips_distance.min()
        min_lips_frame = lips_distance.argmin()
        min_lips_ema = consonant_ema[min_lips_frame]
        
        consonant_metrics["min_lips_distance"].append(min_lips_distance)
        consonant_metrics["min_lips_frame"].append(min_lips_frame)
        consonant_metrics["min_lips_ema"].append(min_lips_ema)
        
        tongue_tip_ema = consonant_ema[:, 2:4]
        tongue_mid_ema = consonant_ema[:, 4:6]
        repeated_palate = np.tile(palate, (nb_frames, 1, 1))
        
        tongue_tip_distances = np.sqrt(((tongue_tip_ema[:, None, :] - repeated_palate) ** 2).sum(axis=-1))
        min_tongue_tip_distance = tongue_tip_distances.min()
        argmin_tongue_tip_distance = tongue_tip_distances.min(axis=1).argmin()
        min_tongue_tip_ema = consonant_ema[argmin_tongue_tip_distance]
        
        consonant_metrics["min_tongue_tip_distance"].append(min_tongue_tip_distance)
        consonant_metrics["min_tongue_tip_ema"].append(min_tongue_tip_ema)
        
        tongue_mid_distances = np.sqrt(((tongue_mid_ema[:, None, :] - repeated_palate) ** 2).sum(axis=-1))
        min_tongue_mid_distance = tongue_mid_distances.min()
        argmin_tongue_mid_distance = tongue_mid_distances.min(axis=1).argmin()
        min_tongue_mid_ema = consonant_ema[argmin_tongue_mid_distance]
        
        consonant_metrics["min_tongue_mid_distance"].append(min_tongue_mid_distance)
        consonant_metrics["min_tongue_mid_ema"].append(min_tongue_mid_ema)

KeyError: 'min_lips_frame'

In [ ]:
distances = ["lips", "tongue_tip", "tongue_mid"]

def show_consonant(consonant):
    consonant_metrics = consonants_metrics[consonant]
    
    plt.figure(dpi=120)
    
    for i_distance, distance in enumerate(distances):
        distance_ema = np.array(consonant_metrics["min_%s_ema" % distance])
        ax = plt.subplot(3, 2, 1 + 2 * i_distance, aspect="equal")
        ax.plot(palate[:, 0], palate[:, 1])
        ax.scatter(distance_ema[:, 0::2], distance_ema[:, 1::2], s=1)
        
        ax = plt.subplot(3, 2, 2 + 2 * i_distance)
        ax.hist(consonant_metrics["min_%s_distance" % distance])
    
    plt.tight_layout()
    plt.show()

ipw.interactive(show_consonant, consonant=TESTED_CONSONANTS)